In [7]:
import sys
if '/home/rossco/immunova' not in sys.path:
    sys.path.append('/home/rossco/immunova')
from immunova.data.fcs_experiments import FCSExperiment
from immunova.flow.gating.actions import Gating
from immunova.flow.gating.transforms import apply_transform
from immunova.flow.gating.defaults import ChildPopulationCollection
from immunova.flow.gating.utilities import centroid, multi_centroid_calculation
from immunova.flow.plotting.static_plots import Plot
from immunova.data.mongo_setup import test_init
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.neighbors import KDTree
from sklearn.cluster import DBSCAN, KMeans
from multiprocessing import Pool, cpu_count
from functools import partial
from itertools import cycle
from datetime import datetime
from matplotlib.patches import Rectangle, Ellipse
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import pymongo
import pandas as pd
import numpy as np
import functools
import warnings
import os
warnings.filterwarnings('ignore')
test_init()

In [8]:
exp = FCSExperiment.objects(experiment_id='ILTIS_T1').get()

In [9]:
sep = Gating(exp, 'sep6')

In [10]:
data = sep.get_population_df('manual_cd8', transform=True)

In [11]:
data.shape

(4623, 19)

In [12]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=60)
clusterer.fit(data[['CD45RA', 'CD27', 'CCR7']])
data['cluster'] = clusterer.labels_

In [13]:
set(clusterer.labels_)

{-1, 0, 1, 2}

In [14]:
def colours(x):
    if x == -1:
        return 'black'
    if x == 0:
        return 'red'
    if x == 1:
        return 'blue'
    if x == 2:
        return 'green'
data['cluster'] = data['cluster'].apply(lambda x: colours(x))

In [15]:
%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib.pyplot as plt
import numpy as np
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data['CD45RA'], data['CD27'], data['CCR7'], c=data['cluster'], alpha=0.5, s=2)

ax.set_xlabel('CD45RA')
ax.set_ylabel('CD27')
ax.set_zlabel('CCR7')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
x_min, x_max, y_min, y_max = 0,0,0,0
visualise_manual_rect(sep, 'manual_cd8', 'CD45RA', 'CCR7', x_min, x_max, y_min, y_max, 
                      xlim=(0,1), ylim=(0,1), transform=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
x_min, x_max, y_min, y_max = 0,0,0,0
visualise_manual_rect(sep, 'manual_cd8', 'CD45RA', 'CD27', x_min, x_max, y_min, y_max, 
                      xlim=(0,1), ylim=(0,1), transform=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …